In [17]:
from binance.client import Client
import pandas as pd
import numpy as np
import talib

In [2]:
api_key = "8AiGAyxlhYQaRpE1s7097hx5sZ12Ogtr8ir9DsyztaD5j24LrI0fEoToDzCI5lle"
api_secret = "VhudTs0HsBVFNdSTghmqfCjUuIXF6rFiXIfROxHIaM71TGgib7NeZ5aOsJUHjI9f"

In [5]:
client = Client(api_key=api_key, api_secret=api_secret)

In [3]:
def get_history(symbol, interval, start=None, end=None):
    if not start:
        start = "1900-01-01"  # Just random old time for maximum candles
    bars = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end, limit=1000)
    
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit="ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time", "Quote Asset Volume", "Number Of Trades", "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace=True)

    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors="coerce")
    return df

In [170]:
df = get_history("BTCUSDT", "1h", "10-01-01")
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...
2022-11-05 10:00:00,21289.82,21321.79,21280.00,21318.14,7646.741530
2022-11-05 11:00:00,21317.47,21427.34,21311.18,21404.63,13279.971880
2022-11-05 12:00:00,21404.63,21420.84,21288.37,21310.59,10962.717040


In [171]:
df.to_csv("test.csv")

In [181]:
df = pd.read_csv("test.csv", parse_dates=["Date"], index_col="Date")
df

,Open,High,Low,Close,Volume
Date,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...
2022-11-05 10:00:00,21289.82,21321.79,21280.00,21318.14,7646.741530
2022-11-05 11:00:00,21317.47,21427.34,21311.18,21404.63,13279.971880
2022-11-05 12:00:00,21404.63,21420.84,21288.37,21310.59,10962.717040


## talib.SMA() == df.rolling()

In [182]:
df["rsi"] = talib.RSI(df.Close, 14)
df["sma1"] = talib.SMA(df.Close, 7)  # Best values for sma crossover strategy
df["sma2"] = talib.SMA(df.Close, 20)
df["24max"] = df.High.rolling(24).max()  # 24 hour max price
df["365max"] = df.High.rolling(24*365).max()
df

,Open,High,Low,Close,Volume,rsi,sma1,sma2,24max,365max
Date,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,NaN,NaN,NaN,NaN,NaN
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,NaN,NaN,NaN,NaN,NaN
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,NaN,NaN,NaN,NaN,NaN
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,NaN,NaN,NaN,NaN,NaN
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-11-05 10:00:00,21289.82,21321.79,21280.00,21318.14,7646.741530,62.602379,21348.964286,21207.9285,21480.65,69000.0
2022-11-05 11:00:00,21317.47,21427.34,21311.18,21404.63,13279.971880,65.959344,21354.305714,21236.2250,21480.65,69000.0
2022-11-05 12:00:00,21404.63,21420.84,21288.37,21310.59,10962.717040,59.685901,21342.975714,21262.5895,21480.65,69000.0


In [183]:
tmp = pd.DataFrame(index=df.index)
tmp["patterns"] = 0
tmp["pos"] = 0
tmp["neg"] = 0
tmp["tmp"] = 0
functions = (talib.CDL2CROWS, talib.CDL3BLACKCROWS, talib.CDL3INSIDE, talib.CDL3LINESTRIKE, talib.CDL3OUTSIDE, talib.CDL3STARSINSOUTH, talib.CDL3WHITESOLDIERS, talib.CDLABANDONEDBABY, talib.CDLADVANCEBLOCK, talib.CDLBELTHOLD, talib.CDLBREAKAWAY, talib.CDLCLOSINGMARUBOZU, talib.CDLCONCEALBABYSWALL, talib.CDLCOUNTERATTACK, talib.CDLDARKCLOUDCOVER, talib.CDLDOJI, talib.CDLDOJISTAR, talib.CDLDRAGONFLYDOJI, talib.CDLENGULFING, talib.CDLEVENINGDOJISTAR, talib.CDLEVENINGSTAR, talib.CDLGAPSIDESIDEWHITE, talib.CDLGRAVESTONEDOJI, talib.CDLHAMMER, talib.CDLHANGINGMAN, talib.CDLHARAMI, talib.CDLHARAMICROSS, talib.CDLHIGHWAVE, talib.CDLHIKKAKE, talib.CDLHIKKAKEMOD, talib.CDLHOMINGPIGEON, talib.CDLIDENTICAL3CROWS, talib.CDLINNECK, talib.CDLINVERTEDHAMMER, talib.CDLKICKING, talib.CDLKICKINGBYLENGTH, talib.CDLLADDERBOTTOM, talib.CDLLONGLEGGEDDOJI, talib.CDLLONGLINE, talib.CDLMARUBOZU, talib.CDLMATCHINGLOW, talib.CDLMATHOLD, talib.CDLMORNINGDOJISTAR, talib.CDLMORNINGSTAR, talib.CDLONNECK, talib.CDLPIERCING, talib.CDLRICKSHAWMAN, talib.CDLRISEFALL3METHODS, talib.CDLSEPARATINGLINES, talib.CDLSHOOTINGSTAR, talib.CDLSHORTLINE, talib.CDLSPINNINGTOP, talib.CDLSTALLEDPATTERN, talib.CDLSTICKSANDWICH, talib.CDLTAKURI, talib.CDLTASUKIGAP, talib.CDLTHRUSTING, talib.CDLTRISTAR, talib.CDLUNIQUE3RIVER, talib.CDLUPSIDEGAP2CROWS, talib.CDLXSIDEGAP3METHODS)
for function in functions:
    tmp["tmp"] = function(df.Open, df.High, df.Low, df.Close)
    tmp["patterns"] = tmp["patterns"] + tmp["tmp"]
    tmp.loc[tmp["tmp"] > 0, "pos"] += 1
    tmp.loc[tmp["tmp"] < 0, "neg"] -= 1
df["patterns"] = tmp["patterns"]
tmp

,patterns,pos,neg,tmp
Date,,,,
2017-08-17 04:00:00,0,0,0,0
2017-08-17 05:00:00,0,0,0,0
2017-08-17 06:00:00,0,0,0,0
2017-08-17 07:00:00,100,1,0,0
2017-08-17 08:00:00,100,1,0,0
...,...,...,...,...
2022-11-05 10:00:00,200,2,0,0
2022-11-05 11:00:00,200,3,-1,0
2022-11-05 12:00:00,0,0,0,0


In [184]:
df

,Open,High,Low,Close,Volume,rsi,sma1,sma2,24max,365max,patterns
Date,,,,,,,,,,,
2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181009,NaN,NaN,NaN,NaN,NaN,0
2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234916,NaN,NaN,NaN,NaN,NaN,0
2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229691,NaN,NaN,NaN,NaN,NaN,0
2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443249,NaN,NaN,NaN,NaN,NaN,100
2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,NaN,NaN,NaN,NaN,NaN,100
...,...,...,...,...,...,...,...,...,...,...,...
2022-11-05 10:00:00,21289.82,21321.79,21280.00,21318.14,7646.741530,62.602379,21348.964286,21207.9285,21480.65,69000.0,200
2022-11-05 11:00:00,21317.47,21427.34,21311.18,21404.63,13279.971880,65.959344,21354.305714,21236.2250,21480.65,69000.0,200
2022-11-05 12:00:00,21404.63,21420.84,21288.37,21310.59,10962.717040,59.685901,21342.975714,21262.5895,21480.65,69000.0,0


In [178]:
# condition1 = df.rsi < 

Date
2020-11-28 03:00:00    1100
2018-04-09 14:00:00    1000
2019-11-29 17:00:00    1000
2020-03-20 03:00:00    1000
2021-09-11 18:00:00    1000
2018-08-11 19:00:00     900
2018-08-13 10:00:00     900
2019-01-22 07:00:00     900
2019-02-09 02:00:00     900
2019-12-26 22:00:00     900
Name: patterns, dtype: int64

In [ ]:
co